In [1]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [2]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca[zł/h]"])
for (i, job) in enumerate(jobs_list):
    c = 0
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(employment_date), end_date, 1)[0]
            c += 1
        else:
            dismissal_date = None

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, birth_date, pesel,
                                                         city, address, phone, email, employment_date,
                                                         dismissal_date, job, wage]

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [3]:
employees_df

,id,imię,nazwisko,data_urodzenia,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca[zł/h]
0,1,Grzegorz,Bocian,31/03/1971,71033113652,Wrocław,ul. K. K. Baczyńskiego 60/42,48454649526,grzegorz.bocian@kuba.pl,2014-01-01,None,menadżer,69.3
1,2,Janusz,Porębski,11/06/1976,76061137751,Wrocław,ul. Młyńska 36/37,48656530109,janusz.porebski@kuba.pl,2014-01-01,None,sprzedawca,49.4
2,3,Tadeusz,Matusiewicz,04/04/1964,64040412337,Wrocław,ul. Proszowska 16/2,48352726257,tadeusz.matusiewicz@kuba.pl,2014-01-01,None,sprzedawca,51.9
5,4,Michał,Sawicki,11/04/1986,86041120833,Świdwin,ul. Lutyków 27,48132858198,michal.sawicki@kuba.pl,2014-01-01,None,mechanik,45.9
13,5,Grzegorz,Seroczyński,01/12/1994,94120181112,Wrocław,ul. Kijanki 41/18,48799139172,grzegorz.seroczynski@kuba.pl,2014-01-01,None,informatyk,42.4
3,6,Karol,Krajewski,30/11/1980,80113087158,Wałbrzych,ul. Bolesława Prusa 83,48402027616,karol.krajewski@kuba.pl,2014-06-11,2016-02-04,sprzedawca,49.5
4,7,Barbara,Kawka,23/11/1974,74112317189,Wrocław,ul. Mjr. Władysława Starzyka 8/32,48375959571,barbara.kawka@kuba.pl,2014-08-24,None,sprzedawca,55.6
6,8,Rafał,Abutidze,18/08/1993,93081888731,Wrocław,ul. Źródlana 14,48390143267,rafal.abutidze@kuba.pl,2015-03-04,None,mechanik,50.8
7,9,Leo,Gardyan,20/07/1967,67072065278,Prudnik,ul. Szkolna 92,48617800284,leo.gardyan@kuba.pl,2015-06-21,None,mechanik,45.0
8,10,Maciej,Koźluk,11/09/1981,81091140136,Wrocław,ul. Westy 54,48430456294,maciej.kozluk@kuba.pl,2015-11-25,None,mechanik,50.0


# Transakcje

In [4]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [5]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [6]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [7]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [8]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [9]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [10]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [11]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [12]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "data_urodzenia", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        pesel, birth_date = random_pesel_date_man(year_lims)
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        pesel, birth_date = random_pesel_date_woman(year_lims)
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, birth_date[-4:])
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, birth_date, sex, pesel, city, address, loyalty_card, phone, email]

In [13]:
clients_df

,id,imię,nazwisko,data_urodzenia,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Michał,Szewczyk,24/04/1973,M,73042457655,Zawiercie,ul. Kwiatowa 58,False,48820619898,a1h1wp4m8opa3@onet.pl
1,2,Agnieszka,Łubian,18/06/1999,F,99061827169,Wrocław,ul. Książęca 27,True,48874291988,agn-lubia@o2.pl
2,3,Konrad,Stana,01/05/1970,M,70050198093,Wrocław,ul. Leśna 2/2,True,48049597713,lx60ij09lm443s0iq@opayq.com
3,4,Paweł,Dubiel,08/11/1977,M,77110895790,Wrocław,ul. Kwiatowa 89/31,False,48656029994,p-dubie77@gmail.com
4,5,Rafał,Kamola,21/04/1972,M,72042195053,Wrocław,ul. Gajowa 97/18,False,48338498561,r-k3288@onet.pl
...,...,...,...,...,...,...,...,...,...,...,...
1527,1528,Aniela,Wawrzecka,09/12/1975,F,75120976560,Grajewo,ul. Jana Kochanowskiego 12/50,False,48868486986,a_wawr75@onet.pl
1528,1529,Amelia,Kasperek,29/03/1993,F,93032979686,Wrocław,ul. Chopina 1/42,False,48547717960,kasperek19939@wp.pl
1529,1530,Aneta,Norek,04/10/2006,F,06300406869,Wrocław,ul. Królewiecka 2/32,False,48917243485,ano06@wp.pl
1530,1531,Martyna,Bielska,01/07/1988,F,88070102967,Elbląg,ul. Sportowa 70/29,False,48337974807,m-bi@gmail.com


In [14]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(66)

# auta

In [15]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [16]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = (i+1 not in car_ids_sell and i+1 in car_ids_buy)
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [17]:
df_samochody = create_car_table(max(car_ids)-1)

In [18]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Opel,Zafira,4,1999,Sedan,False,srebrny,Benzyna,1.9,Manualna,5,5,False
1,2,Volkswagen,Golf,4,2010,SUV,False,czerwony,Diesel,2.2,Manualna,5,5,False
2,3,Jeep,Renegade,4,2005,Sedan,False,biały,Benzyna,1.9,Manualna,5,2,False
3,4,Suzuki,Access 125,2,2011,,False,czarny,Benzyna,0.3,Manualna,0,1,False
4,5,Volkswagen,Passat,4,2012,SUV,False,srebrny,Diesel,3.3,Manualna,3,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,973,Volkswagen,Passat Variant,4,2003,Sedan,False,czerwony,Diesel,2.2,Automatyczna,5,2,True
973,974,Fiat,500,4,2003,Coupe,False,szary,Benzyna,1.0,Manualna,5,2,True
974,975,Opel,Corsa,4,2003,Kombi,False,niebieski,Benzyna,1.2,Manualna,3,5,False
975,976,Volkswagen,Passat Variant,4,2004,Coupe,False,szary,Benzyna,4.5,Automatyczna,3,5,True


# Usługi

In [19]:
print(len(transaction_dates_service))
print(len(car_ids_service))
print(len(client_ids_service))
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_price = rng.random(size=len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

117
117
117


In [20]:
df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['cena'] = service_price # to trzeb apóźniej uzależnić od użytych części
df_services['data_zaksiegowania'] = transaction_dates_service

In [21]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,cena,data_zaksiegowania
0,1,23,44,8,0.555255,2014-02-28
1,2,37,72,10,0.044503,2014-03-30
2,3,49,94,9,0.236232,2014-05-08
3,4,50,96,10,0.370230,2014-05-09
4,5,57,109,4,0.934391,2014-05-18
...,...,...,...,...,...,...
112,113,757,128,4,0.438772,2020-10-20
113,114,960,1505,9,0.501694,2020-11-14
114,115,964,1514,8,0.106214,2020-11-26
115,116,965,1515,9,0.409228,2020-11-29


# Użyte części

In [22]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(23), array([12, 31,  9])), (np.int64(37), array([30, 37])), (np.int64(49), array([51])), (np.int64(50), array([28])), (np.int64(57), array([29])), (np.int64(72), array([21, 38])), (np.int64(88), array([28, 28])), (np.int64(90), array([38])), (np.int64(98), array([37,  0, 18])), (np.int64(133), array([35, 33])), (np.int64(136), array([49])), (np.int64(155), array([43])), (np.int64(158), array([40])), (np.int64(178), array([29, 39, 13])), (np.int64(182), array([57, 34])), (np.int64(185), array([29])), (np.int64(190), array([12])), (np.int64(207), array([30])), (np.int64(222), array([43])), (np.int64(223), array([28])), (np.int64(242), array([53, 57])), (np.int64(251), array([48])), (np.int64(269), array([40, 61])), (np.int64(278), array([ 1, 55])), (np.int64(281), array([59])), (np.int64(283), array([13])), (np.int64(305), array([12])), (np.int64(307), array([ 6, 60, 17])), (np.int64(310), array([42])), (np.int64(315), array([12, 39, 18])), (np.int64(340), array([60])), (np.in

In [47]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
    else:
        attributes = ''
    price = np.round(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)), 3)
    return part['część'], ' '.join(attributes), price

parts = []
for element in all_car_parts_indexes:
    car_id, parts_id = element
    car = df_samochody.loc()[car_id+1]
    for part_id in parts_id:
        parts.append(create_part(part_id, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['część', 'atrybuty', 'cena']).drop_duplicates(['część', 'atrybuty', 'cena'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'część', 'atrybuty', 'cena']]


In [48]:
parts_df

,id,część,atrybuty,cena
0,1,Drzwi lewe tylne,smart forFour szary,392.0
1,2,Zagłówek,smart forFour,12.0
2,3,Drzwi prawe przednie,smart forFour szary,338.0
3,4,Dywan,Nissan Qashqai,24.0
4,5,Pedał hamulca,Nissan Qashqai,99.0
...,...,...,...,...
167,168,Rozrząd,,3195.0
168,169,Tłumik,,92.0
169,170,Silnik,Nissan E-NV200 1.8,2811.0
170,171,Zewnętrzna klamka drzwi,Citroen C3 Picasso,8.0


# Wyposażenie

In [50]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['część']
equipment_quantity = np.ones(eq_len).astype(int) #to później zmienić
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['ilość'] = equipment_quantity
df_equipment['cena_części'] = equipment_price

In [51]:
df_equipment

,id,nazwa,atrybut,ilość,cena_części
0,1,Drzwi lewe tylne,smart forFour szary,1,392.0
1,2,Zagłówek,smart forFour,1,12.0
2,3,Drzwi prawe przednie,smart forFour szary,1,338.0
3,4,Dywan,Nissan Qashqai,1,24.0
4,5,Pedał hamulca,Nissan Qashqai,1,99.0
...,...,...,...,...,...
167,168,Rozrząd,,1,3195.0
168,169,Tłumik,,1,92.0
169,170,Silnik,Nissan E-NV200 1.8,1,2811.0
170,171,Zewnętrzna klamka drzwi,Citroen C3 Picasso,1,8.0
